In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
#loading model (uploaded into the models folder from google drive)
model_path = 'adam1e-6'

model = tf.keras.models.load_model('models/' + model_path + '.keras')
history = pd.read_csv('models/' + model_path + '.csv')

Model Predictions on HAM & BOSQUE

In [ ]:
ham_imgs = np.load("data/HAM_lesions.npz")['data']
ham_labels = np.load("data/ham_labels.npy")
bosque_data = np.load("data/BOSQUE_data.npy", allow_pickle=True)
imgs = bosque_data[0]
labs = bosque_data[1]
skin = bosque_data[2]

#splitting into fair, dark, and especially dark skintones
fair_labels = labs[skin < 4]
fair_imgs = imgs[skin < 4]

dark_labels = labs[skin > 3]
dark_imgs = imgs[skin > 3]

vdark_labels = labs[skin > 4]
vdark_imgs = imgs[skin > 4]

In [ ]:
#evaluting based on uploaded model
pred_fair = np.array(model.predict(fair_imgs)).T[0]
pred_dark = np.array(model.predict(dark_imgs)).T[0]
pred_vdark = np.array(model.predict(vdark_imgs)).T[0]
pred_ham = np.array(model.predict(ham_imgs)).T[0]

#splitting predictions into a confusion matrix: which images were correctly & incorrectly labeled as what
def predictions(true_labs, model_labs):
    out = np.array([[0,0],[0,0]])
    out[0,0] = len(true_labs[model_labs<0.5]) - sum(true_labs[model_labs<0.5]) #correctly benign
    out[0,1] = sum(true_labs[model_labs<0.5]) # incorrectly benign
    out[1,1] = sum(true_labs[model_labs>0.5]) # correctly malignant
    out[1,0] = len(true_labs[model_labs>0.5]) - sum(true_labs[model_labs>0.5]) # incorrectly malignant
    return(out)

res_fair = predictions(fair_labels, pred_fair)
res_dark = predictions(dark_labels, pred_dark)
res_vdark = predictions(vdark_labels,pred_vdark)
res_ham = predictions(ham_labels,pred_ham)


print(res_fair)
print(res_dark)
print(res_vdark)
print(res_ham)

Loss curves

In [ ]:
#printing loss curves for HAM and BOSQUE groups
plt.plot(history['loss'], label = "training", color = 'black')
plt.plot(history['val_loss'], label = "validation", color = 'grey')
plt.plot(history['fair_loss'], label = "fair", color = 'lightblue')
plt.plot(history['dark_loss'], label = "dark", color = 'blue')
plt.plot(history['vdark_loss'], label = "very dark", color = 'darkblue')

plt.ylabel('loss')
plt.xlabel('epoch')
plt.ylim([0,4.5])